In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
import os
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.stats import spearmanr, pearsonr
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA

cwd = os.getcwd()
# print(cwd)

# # for windownoobs
game_price_prediction_path = os.path.abspath(os.path.join(cwd, '..', '..'))

# # for mac
#game_price_prediction_path = os.path.abspath(os.path.join(cwd))

sys.path.insert(0, os.path.abspath(game_price_prediction_path))

from python_scripts.utilities.api_calls import get_cookie_from_blob, fetch_item_to_df, fetch_items
from python_scripts.sentiment_analysis.config import ITEM, ALL_POLARITY_FILENAME

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""" 
- Takes data from mention_data 
- and a range of others; polarity_data, or fetches price history
- Plots it sexily
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""" 

c:\Users\Nukul\Desktop\Code\game_price_prediction\python_scripts\sentiment_analysis


'"" \n- Takes data from mention_data \n- and a range of others; polarity_data, or fetches price history\n- Plots it sexily\n'

### Prerequisites


In [2]:
# Make sure the correct item is imported from config.py

print(f"ITEM = {ITEM}")

ITEM = M4A1-S | Golden Coil (Factory New)


In [3]:
# filter_file_path = os.path.join(game_price_prediction_path, 'python_scripts', 'sentiment_analysis', 'filter_file.py')
# !python "{filter_file_path}"

# mention_counter_path = os.path.join(game_price_prediction_path, 'python_scripts', 'sentiment_analysis', 'mention_counter.py')
# !python "{mention_counter_path}"

# mention_data_combiner_path = os.path.join(game_price_prediction_path, 'python_scripts', 'sentiment_analysis', 'mention_data_combiner.py')
# !python "{mention_data_combiner_path}"

# vader_polarity_path = os.path.join(game_price_prediction_path, 'python_scripts', 'sentiment_analysis', 'vader_polarity.py')
# !python "{vader_polarity_path}"


### Fetch price and volume history

In [4]:
### FETCHING ITEM

# Fetch price and volume data FOR ITEM CHOSEN
dailyCookie = get_cookie_from_blob()
items = fetch_items()
df = fetch_item_to_df(ITEM, dailyCookie)
print(f"Data for {ITEM} fetched")


Data for M4A1-S | Golden Coil (Factory New) fetched


### Plotting polarity (combined) and volume

In [6]:
# Load polarity data instead of mentions
polarity_df = pd.read_csv(os.path.join(game_price_prediction_path, 'data', 'reddit_data', 'polarity_all', ALL_POLARITY_FILENAME))
polarity_df['date'] = pd.to_datetime(polarity_df['date'])

# Ensure df has a 'date' column
if 'date' not in df.columns:
    df = df.reset_index()

# Merge polarity and volume data
merged_df = pd.merge(polarity_df, df[['date', 'volume']], on='date', how='outer').sort_values('date')
merged_df = merged_df.fillna(method='ffill')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['compound'], name="Sentiment", line=dict(color='blue')),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['volume'], name="Volume", line=dict(color='red')),
    secondary_y=True,
)

# Update layout for better readability
fig.update_layout(
    title_text=f"Sentiment and Volume Over Time for {ITEM}",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=50, r=50, t=80, b=50),
)

# Update y-axes
fig.update_yaxes(title_text="Sentiment Score (Compound)", secondary_y=False, gridcolor='lightgrey')
fig.update_yaxes(title_text="Volume Traded", secondary_y=True, gridcolor='lightgrey')

# Update x-axis
fig.update_xaxes(title_text="Date", gridcolor='lightgrey')

# Show the figure
fig.show()

C:\Users\Nukul\AppData\Local\Temp\ipykernel_35848\2286666099.py:11: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

